In [3]:
import pandas as pd
import numpy as np
import json
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Load Yelp reviews data
reviews = []
with open('yelp_academic_dataset_review.json', encoding='utf-8') as f:
    for line in f:
        review = json.loads(line)
        sentiment = 1 if review['stars'] >= 4 else 0
        reviews.append((review['text'], sentiment))

df = pd.DataFrame(reviews, columns=['text', 'sentiment'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jadak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jadak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df.head()

,text,sentiment
0,My wife took me here on my birthday for breakf...,1
1,I have no idea why some people give bad review...,1
2,love the gyro plate. Rice is so good and I als...,1
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",1
4,General Manager Scott Petello is a good egg!!!...,1


In [5]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove HTML tags
    text = re.sub(r"<.*?>", "", text)

    # Remove URLs
    text = re.sub(r"https?://\S+|www\.\S+", "", text)

    # Remove random characters
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (IOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    text = emoji_pattern.sub("", text)

    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Tokenise
    tokens = word_tokenize(text.lower())

    # Remove stopwords
    filtered_tokens = [w for w in tokens if not w in stop_words and w.isalpha()]
    return " ".join(filtered_tokens)

In [6]:
df['processed_text'] = df['text'].apply(preprocess_text)
df.head(2)

,text,sentiment,processed_text
0,My wife took me here on my birthday for breakf...,1,wife took birthday breakfast excellent weather...
1,I have no idea why some people give bad review...,1,idea people give bad reviews place goes show p...


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['processed_text'], df['sentiment'], test_size=0.2, random_state=42)

In [8]:
from collections import Counter

def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

counter = counter_word(df.processed_text)
num_words = len(counter)
print(num_words)
counter

211780


Counter({'wife': 8137,
         'took': 18849,
         'birthday': 4827,
         'breakfast': 17518,
         'excellent': 15638,
         'weather': 2409,
         'perfect': 15191,
         'made': 28693,
         'sitting': 6316,
         'outside': 13890,
         'overlooking': 306,
         'grounds': 991,
         'absolute': 1359,
         'pleasure': 1196,
         'waitress': 9836,
         'food': 139482,
         'arrived': 6845,
         'quickly': 6149,
         'semibusy': 4,
         'saturday': 7051,
         'morning': 6627,
         'looked': 11792,
         'like': 114692,
         'place': 150375,
         'fills': 429,
         'pretty': 40064,
         'earlier': 1626,
         'get': 88257,
         'better': 33278,
         'favor': 1082,
         'bloody': 1107,
         'mary': 914,
         'phenomenal': 1410,
         'simply': 4155,
         'best': 44801,
         'ive': 47266,
         'ever': 25007,
         'im': 53423,
         'sure': 26443,
      

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df.processed_text)
vocab_length = len(tokenizer.word_index)+1

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_pad = pad_sequences(X_train_seq, maxlen=200)

X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=200)

In [23]:
import gensim.downloader as api
pretrained_embedding_model = api.load("fasttext-wiki-news-subwords-300")
# pretrained_embedding_model = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 958.5/958.4MB downloaded


In [17]:
embedding_matrix = np.zeros((vocab_length, 300))

for word, i in tokenizer.word_index.items():
    if word in pretrained_embedding_model:
        embedding_matrix[i] = pretrained_embedding_model[word]

[==================================================] 100.0% 128.1/128.1MB downloaded


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

model = Sequential()

model.add(Embedding(input_dim=vocab_length,
                    output_dim=100,
                    weights=[embedding_matrix],
                    input_length=200,
                    trainable=False))
model.add(GRU(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [19]:
history = model.fit(X_train_pad, y_train, epochs=5, validation_data=(X_test_pad, y_test), batch_size=64)

Epoch 1/5
2874/2874 [==============================] - 302s 104ms/step - loss: 0.4040 - accuracy: 0.8148 - val_loss: 0.3560 - val_accuracy: 0.8412
Epoch 2/5
2874/2874 [==============================] - 259s 90ms/step - loss: 0.3449 - accuracy: 0.8486 - val_loss: 0.3426 - val_accuracy: 0.8474
Epoch 3/5
2874/2874 [==============================] - 262s 91ms/step - loss: 0.3297 - accuracy: 0.8568 - val_loss: 0.3316 - val_accuracy: 0.8547
Epoch 4/5
2874/2874 [==============================] - 249s 87ms/step - loss: 0.3207 - accuracy: 0.8608 - val_loss: 0.3306 - val_accuracy: 0.8547
Epoch 5/5
2874/2874 [==============================] - 276s 96ms/step - loss: 0.3139 - accuracy: 0.8641 - val_loss: 0.3248 - val_accuracy: 0.8575


In [20]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print('Test accuracy:', accuracy)

1437/1437 [==============================] - 34s 24ms/step - loss: 0.3248 - accuracy: 0.8575
Test accuracy: 0.857509434223175


In [10]:
def preprocess_and_predict(text):
    # Preprocess the text
    processed_text = preprocess_text(text)

    # Convert to a sequence
    sequence = tokenizer.texts_to_sequences([processed_text])
    padded_sequence = pad_sequences(sequence, maxlen=200)

    # Predict
    prediction = model.predict(padded_sequence)

    return 'Positive' if prediction[0][0] > 0.5 else 'Negative'

In [ ]:
# Test with a positive review
input_text = "The food was absolutely wonderful, from preparation to presentation, very pleasing."
predicted_sentiment = preprocess_and_predict(input_text)
print(f'Review is: {predicted_sentiment}')

# Test with a negative review
negative_input_text = "Unfortunately, the experience was disappointing. The service was slow and the food was bland. Definitely not worth the price."
negative_predicted_sentiment = preprocess_and_predict(negative_input_text)
print(f'Review is: {negative_predicted_sentiment}')